# 4. Defining new Physics-Based Prognostic Models

All of the past sections describe how to use an existing model. In this section we will describe how to create a new model. This section specifically describes creating a new physics-based model. For training and creating data-driven models see 5. Data-driven Models.

## Linear Models

The easiest model to build is a linear model. Linear models are defined as a linear time series, which can be defined by the following equations:



**The State Equation**:
$$
\frac{dx}{dt} = Ax + Bu + E
$$

**The Output Equation**:
$$
z = Cx + D
$$

**The Event State Equation**:
$$
es = Fx + G
$$

$x$ is `state`, $u$ is `input`, $z$ is `output`, and $es$ is `event state`

Linear Models are defined by creating a new model class that inherits from progpy's LinearModel class and defines the following properties:
* $A$: 2-D np.array[float], dimensions: n_states x n_states. <font color = 'teal'>The state transition matrix. It dictates how the current state affects the change in state dx/dt.</font>
* $B$: 2-D np.array[float], optional (zeros by default), dimensions: n_states x n_inputs. <font color = 'teal'>The input matrix. It dictates how the input affects the change in state dx/dt.</font>
* $C$: 2-D np.array[float], dimensions: n_outputs x n_states. The output matrix. <font color = 'teal'>It determines how the state variables contribute to the output.</font>
* $D$: 1-D np.array[float], optional (zeros by default), dimensions: n_outputs x 1. <font color = 'teal'>A constant term that can represent any biases or offsets in the output.</font>
* $E$: 1-D np.array[float], optional (zeros by default), dimensions: n_states x 1. <font color = 'teal'>A constant term, representing any external effects that are not captured by the state and input.</font>
* $F$: 2-D np.array[float], dimensions: n_es x n_states. <font color = 'teal'>The event state matrix, dictating how state variables contribute to the event state.</font>
* $G$: 1-D np.array[float], optional (zeros by default), dimensions: n_es x 1. <font color = 'teal'>A constant term that can represent any biases or offsets in the event state.</font>
* __inputs__:  list[str] - `input` keys
* __states__:  list[str] - `state` keys
* __outputs__: list[str] - `output` keys
* __events__:  list[str] - `event` keys

We will now utilize our LinearModel to model the classical physics problem throwing an object into the air. This is a common example model, the non-linear version of which (`progpy.examples.ThrownObject`) has been used frequently throughout the examples. This version of ThrownObject will behave nearly identically to the non-linear ThrownObject, except it will not have the non-linear effects of air resistance.

We can create a subclass of LinearModel which will be used to simulate an object thrown, which we will call the ThrownObject Class.

First, some definitions for our Model:

**Events**: (2)
* `falling: The object is falling`
* `impact: The object has hit the ground`

**Inputs/Loading**: (0)
* `None`

**States**: (2)
* `x: Position in space (m)`
* `v: Velocity in space (m/s)`

**Outputs/Measurements**: (1)
* `x: Position in space (m)`

Now, for our keyword arguments:

* <font color = green>__thrower_height : Optional, float__</font>
  * Height of the thrower (m). Default is 1.83 m
* <font color = green>__throwing_speed : Optional, float__</font>
  * Speed at which the ball is thrown (m/s). Default is 40 m/s

With our definitions, we can now create the ThrownObject Model.

First, we need to import the necessary packages.

In [ ]:
import numpy as np
from progpy import LinearModel

Now we'll define some features of a ThrownObject LinearModel. Recall that all LinearModels follow a set of core equations and require some specific properties (see above). In the next step, we'll define our inputs, states, outputs, and events, along with the $A$, $C$, $E$, and $F$ values.

First, let's consider state transition. For an object thrown into the air without air resistance, velocity would decrease literally by __-9.81__ 
$\dfrac{m}{s^2}$ due to the effect of gravity, as described below:

 $$\frac{dv}{dt} = -9.81$$

 Position change is defined by velocity (v), as described below:
 
 $$\frac{dx}{dt} = v$$

 Note: For the above equation x is position not state. Combining these equations with the model $\frac{dx}{dt}$ equation defined above yields the A and E matrix defined below. Note that there is no B defined because this model does not have any inputs.

In [ ]:
class ThrownObject(LinearModel):
    events = ['impact']
    inputs = []  
    states = ['x', 'v']
    outputs = ['x']
    
    A = np.array([[0, 1], [0, 0]])
    C = np.array([[1, 0]])
    E = np.array([[0], [-9.81]])
    F = None

Note that we defined our `A`, `C`, `E`, and `F` values to fit the dimensions that were stated at the beginning of the notebook! Since the parameter `F` is not optional, we have to explicitly set the value as __None__.

Next, we'll define some default parameters for our ThrownObject model.

In [ ]:
class ThrownObject(ThrownObject):  # Continue the ThrownObject class
    default_parameters = {
        'thrower_height': 1.83,
        'throwing_speed': 40,
    }

In the following cells, we'll define some class functions necessary to perform prognostics on the model.

The `initialize()` function sets the initial system state. Since we have defined the `x` and `v` values for our ThrownObject model to represent position and velocity in space, our initial values would be the thrower_height and throwing_speed parameters, respectively.

In [ ]:
class ThrownObject(ThrownObject):
    def initialize(self, u=None, z=None):
        return self.StateContainer({
            'x': self.parameters['thrower_height'],
            'v': self.parameters['throwing_speed']
            })

For our `threshold_met()`, we define the function to return True for event 'falling' when our thrown object model has a velocity value of less than 0 (object is 'falling') and for event 'impact' when our thrown object has a distance from of the ground of less than or equal to 0 (object is on the ground, or has made 'impact').

`threshold_met()` returns a _dict_ of values, if each entry of the _dict_ is __True__, then our threshold has been met!

In [ ]:
class ThrownObject(ThrownObject):
    def threshold_met(self, x):
        return {
            'falling': x['v'] < 0,
            'impact': x['x'] <= 0
        }

Finally, for our `event_state()`, we will calculate the measurement of progress towards the events. We normalize our values such that they are in the range of 0 to 1, where 0 means the event has occurred.

In [ ]:
class ThrownObject(ThrownObject):
    def event_state(self, x): 
        x_max = x['x'] + np.square(x['v'])/(9.81*2)
        return {
            'falling': np.maximum(x['v']/self.parameters['throwing_speed'],0),
            'impact': np.maximum(x['x']/x_max,0) if x['v'] < 0 else 1
        }

With these functions created, we can now use the `simulate_to_threshold()` function to simulate the movement of the thrown object in air. For more information, see 1. Simulation.

In [ ]:
m = ThrownObject()
save = m.simulate_to_threshold(print=True, save_freq=1, threshold_keys='impact', dt=0.1)

__Note__: Because our model takes in no inputs, we have no need to define a future loading function. However, for most models, there would be inputs, and thus a need for a future loading function. For more information on future loading functions and when to use them, please refer to the future loading section in 1. Simulation.

Let's take a look at the outputs of this model

In [ ]:
fig = save.outputs.plot(title='generated model')

Notice that that plot resembles a parabola, which represents the position of the ball through space as time progresses!

For more information on Linear Models, see the [Linear Model](https://nasa.github.io/progpy/api_ref/prog_models/LinearModel.html) Documentation.

## New State Transition Models

In general, new physics-based models are constructed by subclassing [progpy.PrognosticsModel](https://nasa.github.io/progpy/api_ref/prog_models/PrognosticModel.html#prog_models.PrognosticsModel). To create a new model, we'll create a new model class that inherits from this class. Once constructed in this way, the analysis and simulation tools for PrognosticsModels will work on the new model.

For this example, we'll create a simple state-transition model of an object thrown upward into the air without air resistance. Note that this is the same dynamic system as the linear model example above, but formulated in a different way. 

First, we'll import the necessary package to create a general prognostics model.

In [ ]:
from progpy import PrognosticsModel

Next, we'll define our model class. PrognosticsModels require that inputs, states, outputs, and events keys are defined. As in the above example, the states include position (`x`) and velocity(`v`) of the object, the output is position (`x`), and the events are `falling` and `impact`. 

Note that we define this class as `ThrownObject_ST` to distinguish it as a state-transition model compared to the previous linear model class. 

In [ ]:
class ThrownObject_ST(PrognosticsModel):
    """
    Model that simulates an object thrown into the air without air resistance
    """

    inputs = [] # no inputs, no way to control
    states = [
        'x', # Position (m) 
        'v'  # Velocity (m/s)
        ]
    outputs = [ # Anything we can measure
        'x' # Position (m)
    ]
    events = [
        'falling', # Event- object is falling
        'impact' # Event- object has impacted ground
    ]

Next, we'll add some default parameter definitions. These values can be overwritten by passing parameters into the constructor. 

In [ ]:
class ThrownObject_ST(ThrownObject_ST):

    default_parameters = {
        'thrower_height': 1.83, # default height 
        'throwing_speed': 40, # default speed
        'g': -9.81,  # Acceleration due to gravity (m/s^2)
        'process_noise': 0.0  # amount of noise in each step
    }

All prognostics models require some specific class functions. We'll define those next. 

First, we'll need to add the functionality to set the initial state of the system. There are two ways to provide the logic to initialize model state. 

1. Provide the initial state in `parameters['x0']`, or 
2. Provide an `initialization` function 

Here, we'll define an `initialize` function that pulls initial values from the parameters and additionally defines a value for maximum height. 

In [ ]:
class ThrownObject_ST(ThrownObject_ST):

    def initialize(self, u=None, z=None):
        self.max_x = 0  # Set maximum height
        return self.StateContainer({
            'x': self.parameters['thrower_height'],  # Thrown, so initial altitude is height of thrower
            'v': self.parameters['throwing_speed']   # Velocity at which the ball is thrown - this guy is a professional baseball pitcher
            })

Next, the PrognosticsModel class requires that we define how the state transitions throughout time. For continuous models, this is defined with the method `dx`, which calculates the first derivative of the state at a specific time. For discrete systems, this is defined with the method `next_state`, using the state transition equation for the system. 

Here, we use the equations for the derivatives of our system.

In [ ]:
class ThrownObject_ST(ThrownObject_ST):

    def dx(self, x, u):
        return self.StateContainer({
                'x': x['v'], 
                'v': self.parameters['g']})  # Acceleration of gravity

Next, we'll define the `output` method, which will calculate the output given the current state. Here, our output is position. 

In [ ]:
class ThrownObject_ST(ThrownObject_ST):
     
    def output(self, x):
        return self.OutputContainer({'x': x['x']})

The next method we define is `event_state`. As before, 
`event_state` calculates the progress towards the events. Normalized to be between 0 and 1, 1 means there is no progress towards the event and 0 means the event has occurred. 

In [ ]:
class ThrownObject_ST(ThrownObject_ST):
    
    def event_state(self, x): 
        self.max_x = max(self.max_x, x['x'])  # Maximum altitude
        return {
            'falling': max(x['v']/self.parameters['throwing_speed'],0),  # Throwing speed is max speed
            'impact': max(x['x']/self.max_x,0)  # 1 until falling begins, then it's fraction of height
        }

At this point, we have defined all necessary information for the PrognosticsModel to be complete. There are other methods that can additionally be defined (see the [PrognosticsModel](https://nasa.github.io/progpy/api_ref/prog_models/PrognosticModel.html) documentation for more information) to provide further configuration for new models.

As an example of one of these, we additionally define a `threshold_met` equation. Note that this is optional. Leaving `threshold_met` empty will use the event state to define thresholds (threshold = event state == 0). However, this implementation is more efficient, so we include it. 

Here, we define `threshold_met` in the same way as our linear model example. `threshold_met` will return a _dict_ of values, one for each event. Threshold is met when all dictionary entries are __True__. 

In [ ]:
class ThrownObject_ST(ThrownObject_ST):

    def threshold_met(self, x):
        return {
            'falling': x['v'] < 0, # Falling occurs when velocity becomes negative
            'impact': x['x'] <= 0 # Impact occurs when the object hits the ground, i.e. position is <= 0
        }

With that, we have created a new ThrownObject state-transition model. 

Now let's can test our model through simulation. First, we'll create an instance of the model.

In [ ]:
m_st = ThrownObject_ST()

We'll start by simulating to impact. We'll specify the `threshold_keys` to specifically indicate we are interested in impact. 

In [ ]:
# Simulate to impact
event = 'impact'
simulated_results = m_st.simulate_to_threshold(threshold_keys=[event], dt=0.005, save_freq=1, print = True)

# Print result: 
print('The object hit the ground in {} seconds'.format(round(simulated_results.times[-1],2)))

To summarize this section, we have illustrated how to construct new physics-based models by subclassing from [progpy.PrognosticsModel](https://nasa.github.io/progpy/api_ref/prog_models/PrognosticModel.html#prog_models.PrognosticsModel). Some elements (e.g. inputs, states, outputs, events keys; methods for initialization, dx/next_state, output, and event_state) are required. Models can be additionally configured with additional methods and parameters. 

## Derived Parameters

In the previous section, we constructed a new model from scratch, by subclassing from [progpy.PrognosticsModel](https://nasa.github.io/progpy/api_ref/prog_models/PrognosticModel.html#prog_models.PrognosticsModel) and specifying all of the necessary model components. An additional optional feature of PrognosticsModels is derived parameters, and we illustrate this below. 

A derived parameter is a parameter that is a function of another parameter. For example, in the case of a thrown object, one could assume that throwing speed is a function of thrower height, with taller throwing height resulting in faster throwing speeds. When such derived parameters exist, they must be updated whenever the parameters they depend on are updated. In PrognosticsModels, this is achieved with the `derived_params` method. 

For this example, we will use the `ThrownObject_ST` model created in the previous section. We will extend this model to include a derived parameter, namely `throwing_speed` will be dependent on `thrower_height`.

To implement this, we must first define a function for the relationship between the two parameters. We'll assume that `throwning_speed` is a linear function of `thrower_height`. 

In [ ]:
def update_thrown_speed(params):
    return {
        'throwing_speed': params['thrower_height'] * 21.85
    }  
    # Note: one or more parameters can be changed in these functions, whatever parameters are changed are returned in the dictionary

Next, we'll define the parameter callbacks, so that `throwing_speed` is updated appropriately any time that `thrower_height` changes. The following effectively tells the derived callbacks feature to call the `update_thrown_speed` function whenever the `thrower_height` changes. 

In [ ]:
class ThrownObject_ST(ThrownObject_ST):

    param_callbacks = {
        'thrower_height': [update_thrown_speed]
    }

You can also have more than one function be called when a single parameter is changed. You would do this by adding the additional callbacks to the list (e.g., 'thrower_height': [update_thrown_speed, other_fcn])

We have now added the capability for `throwing_speed` to be a derived parameter. Let's try it out. First, we'll create an instance of our class and print out the default parameters. 

In [ ]:
obj = ThrownObject_ST()
print("Default Settings:\n\tthrower_height: {}\n\tthowing_speed: {}".format(obj.parameters['thrower_height'], obj.parameters['throwing_speed']))

Now, let's change the thrower height. If our derived parameters work correctly, the thrower speed should change accordingly. 

In [ ]:
obj.parameters['thrower_height'] = 1.75  # Our thrower is 1.75 m tall
print("\nUpdated Settings:\n\tthrower_height: {}\n\tthowing_speed: {}".format(obj.parameters['thrower_height'], obj.parameters['throwing_speed']))

As we can see, when the thrower height was changed, the throwing speed was re-calculated too. 

In this example, we illustrated how to use the `derived_params` feature, which allows a parameter to be a function of another parameter. 

## Direct Models

## Matrix Models

## State Limits

## Custom Events

## Conclusions